In [1]:
import eikon as ek
import pandas as pd
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
import datetime as dt
from IPython.display import display, HTML
import os
# from sklearn.linear_model import LinearRegression
from statsmodels.regression.linear_model import OLS
from statsmodels.regression.rolling import RollingOLS
from statsmodels.tools.tools import add_constant
import seaborn as sns
from scipy import stats
import time

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [2]:
%run paths.ipynb

In [3]:
%run funcs.ipynb

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 27)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 27)

In [ ]:
file = 'Developed_5_Factors_Daily.csv'

In [ ]:
path_apikeys

In [ ]:
eikon_api = Eikon()


In [ ]:
load_data = False

In [ ]:
indices_rics = ['LVMH.PA', 'ASML.AS', 'NESN.S', 'LIN',
                'NOVOb.CO', 'AAPL.O', 'ROG.S', 'UNH',
                'SAPG.DE', 'MSFT.O']
# indices_rics = eikon_api.get_index_constituents(ek, '.GDAXI')
# indices_rics

In [ ]:
# end_row = 402
fm = FamaFrenchFactors()
df_factors = fm.get_data(file)

In [ ]:
df_factors.tail()

In [ ]:
# for f in df_factors.columns:
#     plot_acf(df_factors[f])
#     plt.show()

In [ ]:
params = {
    'SDate':'1999-12-31',
    'EDate': '2021-06-04',
    'Curn':'Native',
}

if load_data:
    eikon_api.get_timeseries(rics=indices_rics, params=params)
    price_data = eikon_api.data
    price.to_csv(fr'{data_path}\reuters_sample_data.csv')    
else:
    price_data = pd.read_csv(fr'{data_path}\reuters_sample_data.csv')
    

In [ ]:
df = pd.DataFrame(index=eikon_api.data.iloc[0].values[0]['Calc Date'], columns=indices_rics)

In [ ]:
eikon_api.data.iloc[0].values[0]

In [ ]:
df = eikon_api.data.iloc[0].values[0][['Calc Date', 'DailyLogReturns']]
df = df.rename({'DailyLogReturns': 0})
for i in range(1, len(eikon_api.data)):
    df2 = eikon_api.data.iloc[i].values[0][['Calc Date', 'DailyLogReturns']]
    df2 = df2.rename({'DailyLogReturns': i})
    df = pd.merge(df, df2, on='Calc Date')


In [ ]:
df.set_index('Calc Date', inplace=True)

In [ ]:
df.plot()

In [ ]:
price_data = eikon_api.data
price.to_csv(fr'{data_path}\reuters_sample_data.csv')    

In [ ]:
ric = indices_rics[1]
print(ric)
df = data_dict[ric]
df = pd.merge(df, df_factors, left_on='Calc Date', right_index=True, how='left')
df.head()

In [ ]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
for f in factors:
    print(f"{f}: {df.loc[:, ['DailyLogReturns', f]].iloc[0,1]}")

In [ ]:
X = df.loc[:, factors].astype(float)
X = add_constant(X)
y = df.loc[:, 'DailyLogReturns'].astype(float)

In [ ]:
model = OLS(
    endog=y,
    exog=X,
    missing='drop',
    hasconst=True,
)
results = model.fit()

In [ ]:
results.params

In [ ]:
results.pvalues

In [ ]:
sns.histplot(y)

In [ ]:
sns.histplot(results.resid)

In [ ]:
stats.kstat(results.resid)

In [ ]:
stats.jarque_bera(results.resid)

In [ ]:
model_roll = RollingOLS(
    endog=y,
    exog=X,
    missing='drop',
    window=1500 # this is approx. 5 years
)
results_roll = model_roll.fit()

In [ ]:
results_roll.params.join(df['Calc Date']).set_index('Calc Date').plot()

In [ ]:
df